In [1]:
import matplotlib.pyplot as plt 
import SimpleITK as sitk 
import os
from misc import Config
from model import *
%matplotlib qt
os.pathsep = "/"

In [2]:
ELASTIX_PATH = os.path.join('../../elastix/elastix.exe')
TRANSFORMIX_PATH = os.path.join('../../elastix/transformix.exe')
FOLDER_RAW = os.path.join('../../Project/Data')
FOLDER_PREPROCESSED = os.path.join('../../Project/DataPreProcessed')
FOLDER_PARAMETERS = os.path.join('../../Project/parameters')
FOLDER_RESULTS = os.path.join('../../Project/results')

In [3]:
print(FOLDER_RAW)


../../Project/Data


In [4]:
config = Config(
    folder_raw = FOLDER_RAW, 
    folder_preprocessed = FOLDER_PREPROCESSED,
    parameter_folder = FOLDER_PARAMETERS,
    parameters = ["Par0001translation.txt", "Par0001affine.txt", "Par0001bspline16.txt"],
    folder_results = FOLDER_RESULTS,
    seed = 0,
    train_size = 12,
)

In [5]:
model = Model(
    config = config,
    elastix_path = ELASTIX_PATH,
    transformix_path = TRANSFORMIX_PATH,
)

In [6]:
model.load()

Loading Data: 100%|██████████| 15/15 [00:07<00:00,  1.96it/s]


In [7]:
model.preprocess_and_save(downscale_factor=4, normalize="none")

Saving Preprocessed Data: 100%|██████████| 15/15 [00:01<00:00, 12.24it/s]


In [8]:
model.run_registration()

Running with parameters ['Par0001translation.txt', 'Par0001affine.txt', 'Par0001bspline16.txt']


  0%|          | 0/3 [00:00<?, ?it/s]

['../../Project/parameters\\Par0001translation.txt', '../../Project/parameters\\Par0001affine.txt', '../../Project/parameters\\Par0001bspline16.txt']
../../elastix/elastix.exe -f ../../Project/DataPreProcessed\p107\mr_bffe.mhd -m ../../Project/DataPreProcessed\p125\mr_bffe.mhd -p ../../Project/parameters\Par0001translation.txt -p ../../Project/parameters\Par0001affine.txt -p ../../Project/parameters\Par0001bspline16.txt -out ../../Project/results\23-03-03_16-00-03\p107\p125


  0%|          | 0/3 [00:00<?, ?it/s]


ElastixError: Elastix crashed with code 1 for command '../../elastix/elastix.exe -f ../../Project/DataPreProcessed\p107\mr_bffe.mhd -m ../../Project/DataPreProcessed\p125\mr_bffe.mhd -p ../../Project/parameters\Par0001translation.txt -p ../../Project/parameters\Par0001affine.txt -p ../../Project/parameters\Par0001bspline16.txt -out ../../Project/results\23-03-03_16-00-03\p107\p125'.

In [ ]:
model.threshold_staple = 0.95
model.run_staple()

Running for parameters BSpline.txt


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model.scores

[[0.6653657401941012, 0.5464058137942353, 0.7694382923219628]]

In [ ]:
dice_score(model.segmentations[0][0]>0.99999999999, model.ground_truths[0], eps=1e-5)

0.7432233910951942

In [ ]:
im = sitk.GetArrayFromImage(sitk.ReadImage(FOLDER_PREPROCESSED + "/p108/mr_bffe.mhd"))
mask = sitk.GetArrayFromImage(sitk.ReadImage(FOLDER_PREPROCESSED + "/p108/prostaat.mhd"))

registered = sitk.GetArrayFromImage(sitk.ReadImage(FOLDER_RESULTS + "/BSpline/p108_as_fixed/p102_as_moving/result.0.mhd"))
transformed = sitk.GetArrayFromImage(sitk.ReadImage(FOLDER_RESULTS + "/BSpline/p108_as_fixed/p102_as_moving/result.mhd"))

In [ ]:
view = lambda x, ax : ScrollView(x).plot(ax, vmin=x.min(), vmax=x.max())

In [ ]:
a = sitk.GetArrayFromImage(res[0]) > 0.95
fig, ax = plt.subplots(1,1)
view(a, ax)